In [2]:
import os 

try:    
    import folium
except ImportError:
    !pip install --user folium
    import folium
    
from folium import plugins
import pandas as pd
import numpy as np

## Data Prep

In [49]:
data=pd.read_csv("/home/dario/NeoMeetup/Visualizations/map/export_european_cities.csv")
data.head()
data.sort_values("ID_event").head()
print(len(data))
data['g_lat']=pd.to_numeric(data.g_lat, errors='coerce')
print(data['g_lat'].isna().sum())
data['g_lat']=data['g_lat'].dropna()

data['g_lon']=pd.to_numeric(data.g_lon, errors='coerce')
print(data['g_lon'].isna().sum())
data['g_lon']=data['g_lon'].dropna()
print(len(data))

133104
68
34
133104


In [4]:
for line in data.itertuples():
    if data.iloc[line.Index]['dist']==0:
        data['dist'].at[line.Index]=7 #arbitrary for milan
data.head()   

,ID_member,ID_event,event_name,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist
0,268442147,msxvfqyzgbdc,Tandem Language Exchange!,Roma,41.9,12.48,"REC23 ""Restaurant Emporio Club"" (lungotevere...",41.882687,12.476001,7.0
1,268442147,259295022,[TALK INTERATTIVO] Scrum 101: una introduzione...,Roma,41.9,12.48,TIM WCAP - Roma,41.882294,12.467431,7.0
2,268442147,260216317,Group Counseling & Emotional intelligence,Roma,41.9,12.48,"Via delle Fratte di Trastevere, 48",41.887978,12.471672,7.0
3,253766803,260516064,Intro to TensorFlow 2 Codelab - TFxR,Roma,41.9,12.48,Sourcesense S.p.A.,41.828033,12.474345,7.0
4,278938809,260052645,Google Home e IFTTT - Meetup #AperiTech di Vox...,Roma,41.9,12.48,LUISS ENLABS,41.902782,12.496366,7.0


In [5]:
data.sort_values(["ID_event","dist"], inplace=True)
data

,ID_member,ID_event,event_name,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist
19992,218084573,106866582,Envie d\u00eatre form\u00e9 \u00e0 la M\u00e9d...,Paris,48.86,2.34,NaN,NaN,NaN,7.000000
35936,278846290,115197702,Carlinada en Madrid - Pug Playtime in Madrid,Madrid,40.42,-3.71,Parque de El Retiro,40.419758,-3.688406,7.000000
73305,278257631,123953142,Game Development Workshop Series - Session 05,London,51.55,-0.17,NaN,NaN,NaN,5.886219
124222,12879421,128105672,Dinner and Drinks Cruise @ IJ Amsterdam,Amsterdam,52.37,4.89,"Cafetaria ""Subway"" ( Central Station Amsterdam...",52.381172,4.895554,7.000000
76803,139972652,131523832,Work in Fashion? Love Fashion?,London,51.5,-0.19,NaN,NaN,NaN,7.939657
13073,55734112,150138502,Network and Graph Visualisation avec Cytoscape,Paris,48.86,2.34,NaN,NaN,NaN,874.785710
3482,60378,16537690,dance and music activities for under 4 year olds,Paris,48.86,2.34,NaN,NaN,NaN,7.000000
115303,91455112,168469672,Talk: Artificial Neural Networks,Berlin,52.52,13.38,NaN,NaN,NaN,7.000000
65572,277957260,175163972,Tutorial on Installing means of resisting surv...,London,51.52,-0.10,NaN,NaN,NaN,81.525620
27894,223071391,195823772,Joyas y Rarezas Cin\u00e9filas,Madrid,40.42,-3.71,Peque\u00f1o Cine Estudio,40.432446,-3.705431,7.000000


In [6]:
ddrop=data.drop_duplicates(subset="ID_event" , keep="last")
ddrop=ddrop.drop("ID_member", axis=1)
#ddrop.sort_values("ID_event")
ddrop=ddrop.reset_index()
ddrop.head()

,index,ID_event,event_name,g_city,g_lat,g_lon,venue_name,v_lat,v_lon,dist
0,19992,106866582,Envie d\u00eatre form\u00e9 \u00e0 la M\u00e9d...,Paris,48.86,2.34,NaN,NaN,NaN,7.000000
1,35936,115197702,Carlinada en Madrid - Pug Playtime in Madrid,Madrid,40.42,-3.71,Parque de El Retiro,40.419758,-3.688406,7.000000
2,73305,123953142,Game Development Workshop Series - Session 05,London,51.55,-0.17,NaN,NaN,NaN,5.886219
3,124222,128105672,Dinner and Drinks Cruise @ IJ Amsterdam,Amsterdam,52.37,4.89,"Cafetaria ""Subway"" ( Central Station Amsterdam...",52.381172,4.895554,7.000000
4,76803,131523832,Work in Fashion? Love Fashion?,London,51.5,-0.19,NaN,NaN,NaN,7.939657


In [ ]:
d=data.groupby("ID_event")['dist'].quantile(0.75).reset_index()
print(d.head())
d.rename(columns={'dist':'maxdist'}, inplace=True)
ddrop=ddrop.merge(d, on="ID_event")
ddrop.head()

In [29]:
d1=ddrop.g_lat
d1.apply(float)

ValueError: could not convert string to float: '4bid Gallery @ OT301, '

## Let's build a map

In [8]:
#legend template
from branca.element import Template, MacroElement

template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: fixed; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legend (draggable!)</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:red;opacity:0.7;'></span>Bacino d'utenza</li>
    <!-- questo è un commento <li><span style='background:orange;opacity:0.7;'></span>Medium</li>
    <li><span style='background:green;opacity:0.7;'></span>Small</li> -->

  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""



macro = MacroElement()
macro._template = Template(template)

html = """
    <h3> Event ID  : </h3><p>{}</p>
    <h3> Event Name: </h3><p>{}</p>
    <h3> Location: </h3><p>{}</p>
    """

legend_html = """
     <div style=”position: fixed; 
     bottom: 50px; left: 50px; width: 100px; height: 90px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     “>&nbsp; Cool Legend <br>
     &nbsp; East &nbsp; <i class=”fa fa-map-marker fa-2x”
                  style=”color:green”></i><br>
     &nbsp; West &nbsp; <i class=”fa fa-map-marker fa-2x”
                  style=”color:red”></i>
      </div>
     """
#m.get_root().add_child(macro)

#m

In [15]:
import branca #brrr
#m1 = folium.Map(location=[20,0], tiles="Mapbox Bright", zoom_start=2)
from folium.plugins import MarkerCluster
from folium.features import DivIcon

m = folium.Map(location=[ddrop['v_lat'].mean(),ddrop['v_lon'].mean()], zoom_start=12)
mc=MarkerCluster()

for line in ddrop.itertuples():
    #try:
        if str(line.v_lat) != "nan" : #where exists use venue lat/lon
            lat=line.v_lat
        else: #elsewhere use group lat
            lat=line.g_lat
            #print("g_lat: ",lat)
        if str(line.v_lon) != "nan": #where exists use venue lat/lon
            lon=line.v_lon
        else: #elsewhere use group lat
            lon=line.g_lon

        f = branca.element.Figure()
        m1=folium.Map([float(lat), float(lon)], zoom_start=10) #eventualmente aggiungere il livello di zoom in base al raggio può essere fyco

        iframe = branca.element.IFrame(
            html=html.format(ddrop['ID_event'].at[line.Index], ddrop['event_name'].at[line.Index], ddrop['venue_name'].at[line.Index]), 
            width=400, height=240)
        popup1 = folium.Popup(iframe, show=True)


        m1.add_child(folium.Circle(
          location=[float(lat), float(lon)],
          popup=popup1,
          radius=int(ddrop.iloc[line.Index]['maxdist'])*1000,
          #radius=1,
          color='crimson',
          fill=False,
          #fill_color='crimson',

        ))
        #print (lat,"   ", lon)
        #trying to add some text
        #m1.add_child(folium.Marker(
            #location=[45.5,8.5],#[lat, lon],
            #icon=DivIcon(icon_size=(150,36), icon_anchor=(0,0),html='<div style="font-size: 24pt">Bacino d\'utenza</div>')
            #))

        #macro1 = MacroElement()
        #macro1._template = Template(template)
        #m1.get_root().add_child(macro1)

        f.html.add_child(branca.element.Element("<h1>Bacino d'utenza</h1>"))
        #f.html.add_child(branca.element.Element(legend_html))

        #fig = folium.element.Figure()
        #fig.html.add_child(folium.element.Element("<h1>This is a title</h1>"))

        #m1.title = 'My fancy title here'
        #f.add_child(m1)
        m1.add_to(f)

        #iframe = branca.element.IFrame(html=html.format(ddrop['event_id'].at[line.Index], ddrop['city'].at[line.Index]), width=500, height=300)
        #popup = folium.Popup(iframe)
        iframe = branca.element.IFrame(width=1000, height=600)
        f.add_to(iframe)

        # Let's put the IFrame in a Popup
        popup = folium.Popup(iframe, max_width=2650)
        mc.add_child(
            folium.Marker(
            location = [lat, lon], 
            popup=popup,
            ))
        #if line.Index is 10:
        #    break
    #except Exception as e:
    #    print(e)
m.add_child(mc)
plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(m)


macro = MacroElement()

macro._template = Template(template)

m.get_root().add_child(macro)
#map.get_root().add_child(macro1)

folium.LayerControl().add_to(m)

display(m)
    
#Save it as html
m.save('/home/dario/NeoMeetup/Visualizations/map/export_milano.html')

RecursionError: maximum recursion depth exceeded in comparison